In [1]:
import pandas as pd
import numpy as np
from os.path import join
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
paths = [
    '../output/webface-full-celeba-simple-cnn.aligned.margin32.grey.flipped.s20.model.h5.x.npy',
    '../output/webface-full-simple-cnn.aligned.margin32.s20.model.h5.x.npy',
    '../output/celeba-full-simple-cnn.aligned.margin32.flipped.s20.model.h5.x.npy',
    
    '../output/webface-full-celeba-simple-cnn.aligned.margin16.grey.flipped.s20.model.h5.x.npy',
    '../output/webface-full-simple-cnn.aligned.margin16.s20.model.h5.x.npy',
    '../output/celeba-full-simple-cnn.aligned.margin16.flipped.s20.model.h5.x.npy',
    
    '../output/webface-full-celeba-simple-cnn.aligned.margin4.grey.flipped.s20.model.h5.x.npy',
    '../output/webface-full-simple-cnn.aligned.margin4.s20.model.h5.x.npy',
    '../output/celeba-full-simple-cnn.aligned.margin4.flipped.s20.model.h5.x.npy',
    
    '../output/webface-full-celeba-simple-cnn.aligned.margin42.grey.flipped.s20.model.h5.x.npy',
    '../output/webface-full-simple-cnn.aligned.margin42.s20.model.h5.x.npy',
    '../output/celeba-full-simple-cnn.aligned.margin42.flipped.s20.model.h5.x.npy',
]

In [3]:
# '../output/webface-full-celeba-simple-cnn.aligned.margin32.grey.flipped.s20.model.h5.x.npy',
# '../output/webface-full-simple-cnn.aligned.margin32.s20.model.h5.x.npy',
# 0.080167	0.011866	0.032167	0.001730

# 3
# 0.081167	0.013355	0.048889	0.001846

# 4
# 0.078667	0.012667	0.037815	0.001258

# 5
# 0.080500	0.010489	0.040815	0.001090

# 6
# 0.077500	0.007932	0.021481	0.001418

# 9
# 0.077333	0.013233	0.036907	0.001253

# 12 with max deep 3
# 0.075500	0.013018	0.041907	0.001946
# with early stop 50
# 0.072667	0.013064	0.000000	0.000000
# with distance
# 	0.070833	0.012001	0.014370	0.000904

In [4]:
X = np.hstack([np.load(p) for p in paths])

In [5]:
X.shape

(6000, 180)

In [6]:
y = np.load('../output/y')
y.shape

(6000,)

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

In [8]:
import xgboost as xgb
params = {}
params["objective"] = "binary:logistic"
params['eval_metric'] = ['logloss', 'error']
# params["eta"] = 0.02
# params["subsample"] = 0.7
# params["min_child_weight"] = 1
# params["colsample_bytree"] = 0.7
params["max_depth"] = 3
params["seed"] = 42

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(params, d_train, 500, watchlist, early_stopping_rounds=100, verbose_eval=100)

[0]	train-logloss:0.501419	train-error:0.077593	valid-logloss:0.506332	valid-error:0.09
Multiple eval metrics have been passed: 'valid-error' will be used for early stopping.

Will train until valid-error hasn't improved in 100 rounds.
[100]	train-logloss:0.037583	train-error:0.002407	valid-logloss:0.170746	valid-error:0.063333
Stopping. Best iteration:
[21]	train-logloss:0.126642	train-error:0.048519	valid-logloss:0.155456	valid-error:0.056667



In [9]:
d_all = xgb.DMatrix(X, label=y)

xgb.cv(params, d_all, 500, nfold=10, metrics=['error'], shuffle=False, early_stopping_rounds=100, seed=42)

,test-error-mean,test-error-std,train-error-mean,train-error-std
0,0.086500,0.009142,0.075926,0.000872
1,0.083500,0.010500,0.070426,0.001761
2,0.075500,0.010410,0.065796,0.001591
3,0.075167,0.008707,0.062778,0.001584
4,0.075000,0.009574,0.061741,0.002602
5,0.072667,0.009638,0.060278,0.001028
6,0.072500,0.010756,0.059556,0.001328
7,0.072167,0.012066,0.058981,0.001585
8,0.073000,0.012129,0.057908,0.001393
9,0.072167,0.011548,0.057630,0.001141
